### Building A Chatbot
In this project, We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.


Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for :
- Conversational RAG : Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take acions

This Project will cover the basic which will be helpful for those two more advance topics.

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_KEY")


In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000139E64049E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000139E6406FF0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Vicky and I am a Cheif AI Engineer")])

AIMessage(content="Hello Vicky! It's nice to meet you. \n\nThat's quite a title!  As a Chief AI Engineer, I imagine you're involved in some fascinating work.  What are some of the most exciting projects you're currently working on?  \n\nI'm always eager to learn more about the applications of AI. \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 23, 'total_tokens': 98, 'completion_time': 0.136363636, 'prompt_time': 0.000212419, 'queue_time': 0.015262952, 'total_time': 0.136576055}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-02cc079b-296b-411c-8d38-81a865a02fd1-0', usage_metadata={'input_tokens': 23, 'output_tokens': 75, 'total_tokens': 98})

In [8]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, My name is Vicky and I am a Cheif AI Engineer"),
    AIMessage(content="Hello Vicky! It's nice to meet you. \n\nThat's quite a title!  As a Chief AI Engineer, I imagine you're involved in some fascinating work. What are some of the most exciting projects you're currently working on?  \n\nI'm always eager to learn more about the applications of AI. \n\n"),
    HumanMessage(content="Hey What is my name and what do I do?")
])


AIMessage(content="According to our conversation, your name is Vicky and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to know or discuss?  😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 115, 'total_tokens': 153, 'completion_time': 0.069090909, 'prompt_time': 0.00362316, 'queue_time': 0.011676189, 'total_time': 0.072714069}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a93643aa-57a2-4f98-ab65-c564cdcce0be-0', usage_metadata={'input_tokens': 115, 'output_tokens': 38, 'total_tokens': 153})

In [9]:
!pip install langchain_community

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ------------- -------------------------- 0.8/2.4 MB 2.2 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 1.8 MB/s eta 0:00:01
   -------------------------- ------------- 1.6/2.4 MB 1.8 MB/s eta 0:00:01
   ----------------------------------- ---- 2.1/2.4 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.1 MB 3.1 MB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.1 MB 3.5 MB/s eta 0:00:01
   -------------------------------------

### Message History
We can use a message History class to wrap out model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastores. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [11]:
store={}

def get_seesion_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_seesion_history)

In [12]:
config = {"configurable":{"session_id":"chat1"}}


In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is vicky and I am a chief AI Engineer")],
    config=config
)

In [14]:
response

AIMessage(content="Hi Vicky, it's great to meet you! That's a fantastic title.  \n\nWhat kind of exciting AI projects are you working on these days?  I'm always eager to learn about the innovative things people are doing with AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 23, 'total_tokens': 77, 'completion_time': 0.098181818, 'prompt_time': 0.000207219, 'queue_time': 0.01411014, 'total_time': 0.098389037}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fac00f1d-9aa3-479d-819e-1e4c25b1d1ed-0', usage_metadata={'input_tokens': 23, 'output_tokens': 54, 'total_tokens': 77})

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my my name?")],
    config=config
)

In [16]:
response

AIMessage(content='Your name is Vicky!  I remember you told me at the beginning of our conversation. 😊 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 92, 'total_tokens': 126, 'completion_time': 0.061818182, 'prompt_time': 0.003488628, 'queue_time': 0.010014532999999999, 'total_time': 0.06530681}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9958903-c69c-4693-98b0-46a38e8021e5-0', usage_metadata={'input_tokens': 92, 'output_tokens': 34, 'total_tokens': 126})

In [17]:
## Change the config --> session id
config2 ={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my my name?")],
    config=config2
)
response.content

"As a large language model, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊 \n\n"

### Prompt templates
Prompt Templates helps to turn raw user information into a format that the LLM can work with. In this  case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, Let's add in a system message with some custom instructions(but still taking messages as input). Next, we'll add in more input besides just the messages.

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is vicky")]})

AIMessage(content="Hello Vicky!  \n\nIt's nice to meet you.  \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 33, 'total_tokens': 62, 'completion_time': 0.052727273, 'prompt_time': 0.000356599, 'queue_time': 0.013020609999999998, 'total_time': 0.053083872}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9db83d03-353c-48ea-86d2-d8a17db69363-0', usage_metadata={'input_tokens': 33, 'output_tokens': 29, 'total_tokens': 62})

In [21]:
with_message_history = RunnableWithMessageHistory(chain, get_seesion_history)

In [22]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is vicky")],
    config=config
)

In [24]:
response.content

"Hi Vicky! It's nice to meet you.  \n\nWhat can I do for you today? 😊  I'm ready for any questions you have. \n"

In [25]:
## Add more complexity
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the nest of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [27]:
response = chain.invoke({
    "messages":[HumanMessage(content="Hi My name is vicky")],
    "language":"Hindi"
    })
response.content

'नमस्ते विक्की! 😊  मुझे बहुत खुशी है कि आप मेरे साथ बात कर रहे हैं।\n\nक्या मैं आपकी कोई मदद कर सकता हूँ? \n'

Let's now wrap this more complicated chain in a messages History class. THis time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [28]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_seesion_history,
    input_messages_key="messages"
)

In [29]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi My name is vicky")],
    "language":"Hindi"
    },
    config=config
)
response.content

'नमस्ते विक्की!  मैं आपकी मदद करने के लिए यहाँ हूँ। \n\nआप क्या जानना चाहते हैं? 😊 \n'

### Managing the conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therfore, it is important to add a step that limits the size of messages you are passing in.


'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.



In [35]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=70,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you're good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2+2?"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes"),
]

In [40]:
!pip install transformers

In [39]:
trimmer.invoke(messages)

[SystemMessage(content="you're good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes', additional_kwargs={}, response_metadata={})]

In [43]:
from operator import  itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")| trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What ice cream do i like")],
        "language":"English"
    }
)
response.content

'You said you like vanilla ice cream! 🍦  \n\nDo you want to talk about ice cream more?  Maybe your favorite toppings?\n'

In [44]:
response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What math problem did i ask")],
        "language":"English"
    }
)
response.content



'You asked "what is 2+2?". \n'

In [ ]:
# Lets wrap this in the message History

with_message_history = RunnableWithMessageHistory(
    chain,
    get_seesion_history,
    input_messages_key="messages"
)

config = {"configurable":{"session_id":"chat5"}}